In [ ]:
### Spawn Player's Car

import carla
import random

# Connect to CARLA simulator
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()

# Get vehicle blueprints and select one randomly
blueprint_library = world.get_blueprint_library()
vehicle_blueprints = blueprint_library.filter('vehicle.*')
vehicle_blueprint = random.choice(vehicle_blueprints)

# Get spawn points and choose a clear one
spawn_points = world.get_map().get_spawn_points()

def find_clear_spawn_point():
    for point in spawn_points:
        waypoint = world.get_map().get_waypoint(point.location)
        if not waypoint.is_junction:
            return point
    return spawn_points[0]

spawn_point = find_clear_spawn_point()
player_car = world.spawn_actor(vehicle_blueprint, spawn_point)

# Adjust spectator camera
spectator = world.get_spectator()
spectator_transform = carla.Transform(
    spawn_point.location + carla.Location(x=-8, z=5),
    spawn_point.rotation
)
spectator.set_transform(spectator_transform)

print(f"Player's car spawned: {vehicle_blueprint.id}")


In [ ]:
### Spawn Traffic

import time

def spawn_traffic(num_vehicles=10):
    traffic_vehicles = []
    for _ in range(num_vehicles):
        spawn_point = random.choice(spawn_points)
        if spawn_point:
            blueprint = random.choice(vehicle_blueprints)
            traffic_vehicle = world.try_spawn_actor(blueprint, spawn_point)
            if traffic_vehicle:
                traffic_vehicle.set_autopilot(True)
                traffic_vehicles.append(traffic_vehicle)
    return traffic_vehicles

traffic_vehicles = spawn_traffic(30)
print(f"Spawned {len(traffic_vehicles)} traffic vehicles.")


In [ ]:
### Enable Autonomous Navigation for Player's Car

import math

def get_next_waypoint(vehicle):
    location = vehicle.get_transform().location
    waypoint = world.get_map().get_waypoint(location, lane_type=carla.LaneType.Driving)
    if waypoint:
        next_waypoints = waypoint.next(2.0)
        return next_waypoints[0] if next_waypoints else None
    return None

def navigate_autonomously(player_car):
    while True:
        waypoint = get_next_waypoint(player_car)
        if not waypoint:
            break
        
        # Calculate direction
        vehicle_transform = player_car.get_transform()
        vehicle_location = vehicle_transform.location
        waypoint_location = waypoint.transform.location
        dx = waypoint_location.x - vehicle_location.x
        dy = waypoint_location.y - vehicle_location.y
        angle = math.atan2(dy, dx)

        # Control car
        control = carla.VehicleControl()
        control.throttle = 0.5
        control.steer = max(-1.0, min(1.0, angle))
        player_car.apply_control(control)

        time.sleep(0.1)

# Start autonomous driving
navigate_autonomously(player_car)


In [ ]:
### Slow Down Based on Distance to Nearest Vehicle and Change Route Color

def get_distance_to_nearest_vehicle(vehicle):
    actors = world.get_actors().filter('vehicle.*')
    distances = [
        vehicle.get_location().distance(actor.get_location())
        for actor in actors if actor.id != vehicle.id
    ]
    return min(distances) if distances else float('inf')

def monitor_distance_and_color_route(player_car):
    while True:
        distance = get_distance_to_nearest_vehicle(player_car)
        if distance < 1.0:
            control = carla.VehicleControl(throttle=0.0, brake=1.0)
            color = carla.Color(255, 0, 0)  # Red
        else:
            control = carla.VehicleControl(throttle=0.5, brake=0.0)
            color = carla.Color(0, 255, 0)  # Green
        
        player_car.apply_control(control)
        waypoint = get_next_waypoint(player_car)
        if waypoint:
            world.debug.draw_point(waypoint.transform.location, size=0.3, color=color, life_time=0.1)
        
        time.sleep(0.1)

monitor_distance_and_color_route(player_car)


In [ ]:
### Display Vehicle Stats on Camera Feed

import cv2
import numpy as np

# Attach a camera to the player's car
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=player_car)

def process_image(image):
    # Convert image to numpy array
    array = np.frombuffer(image.raw_data, dtype=np.uint8).reshape((image.height, image.width, 4))
    image_bgr = cv2.cvtColor(array, cv2.COLOR_BGRA2BGR)
    
    # Display stats
    speed = 3.6 * math.sqrt(player_car.get_velocity().length_squared())  # Convert m/s to km/h
    cv2.putText(image_bgr, f"Speed: {speed:.2f} km/h", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    # Show image
    cv2.imshow('Player Camera', image_bgr)
    cv2.waitKey(1)

camera.listen(lambda image: process_image(image))

try:
    while True:
        time.sleep(0.1)
finally:
    camera.destroy()
    cv2.destroyAllWindows()



In [20]:
### Destruction

import carla

# Connect to the CARLA Simulator
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)  # Set timeout to avoid blocking issues
world = client.get_world()  # Get the current world

# Destroy all vehicles and sensors in the world
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()

for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

print("All vehicles and sensors have been destroyed.")


All vehicles and sensors have been destroyed.
